<p style="text-align:center;">
    <img src="https://raw.githubusercontent.com/skyplane-project/skyplane/main/docs/_static/logo-light-mode.png" width=500>
</p>

# Welcome to Skyplane!

## Skyplane enables fast data transfers between any cloud

Skyplane is a tool for blazingly fast bulk data transfers between object stores in the cloud. It provisions a fleet of VMs in the cloud to transfer data in parallel while using compression and bandwidth tiering to reduce cost.

Skyplane is:
1. 🔥 Blazing fast ([110x faster than AWS DataSync](https://skyplane.org/en/latest/benchmark.html))
2. 🤑 Cheap (4x cheaper than rsync)
3. 🌐 Universal (AWS, Azure, GCP, IBMCloud, and Cloudflare R2)

You can use Skyplane to transfer data: 
* between object stores within a cloud provider (e.g. AWS us-east-1 to AWS us-west-2)
* between object stores across multiple cloud providers (e.g. AWS us-east-1 to GCP us-central1)
* between local storage and cloud object stores (experimental)

# Exercises

This notebook consists of 4 exercises:

1. Exercise 1: Copying data with AWS 
2. Exercise 2: Copying data with Skyplane 
3. Exercise 3: Copying data to multiple destinations (multicast) 
5. Exercise 4: Cleanup 

# Learning outcomes 🎯

After completing this notebook, you would have:

1. An understand the Skyplane API
2. Transfered data for a ML model from AWS S3 object stores in US-EAST-1 (N. Virginia) to EU-WEST-1 (Ireland)
3. Compare and contrast `aws s3 cp` with Skyplane
4. Terminate the transfer and clean up state



# How to use this Tutorial

These notebooks serve as a guide to Skyplane. At any point if you happen to get stuck, feel free to ping us on `#skyplane` channel on the [Skycamp slack.](https://join.slack.com/t/skycamp2023/shared_invite/zt-25axzytwn-y5AR~Bx2nqm4Iec6jlq3JA)


### Let's ensure we have the latest notebook

In [ ]:
# Please run this cell
!git pull --quiet

### Update to the latest Skyplane pip package

In [ ]:
!pip uninstall -y skyplane

In [ ]:
# Please run this cell
!pip install -U "git+https://github.com/skyplane-project/skyplane.git@skycamp-2023#egg=skyplane[aws]"

In [ ]:
!pip install ipywidgets

### Configure Skyplane with AWS credentials.


# Transferring Data with Skyplane

<p style="text-align:center;">
    <img src="./assets/unicast.jpg" width=700>
</p>

The core of Skyplane is based around the `cp` command. Suppose you want to transfer a fine-tuned [Gorilla](https://github.com/ShishirPatil/gorilla) model from one region to another to be accessible to a cross-regional serving cluster. Skyplane can help you efficiently transfer this data so you model weights are accessible accross multiple regions.  Let’s prepare for a transfer by first initializing buckets in a few different cloud regions in AWS.

# Creating Buckets

### Setting up AWS credentials 

💻 `aws configure`
```
AWS Access Key ID [****************ZAWO]: 
AWS Secret Access Key [****************WtGD]: 
Default region name [us-east-1]: 
Default output format [json]: 
```

### Setting up AWS in the Destination region

First, let’s create a bucket in the destination region `aws:ap-south-1` to store the model weights. 

> **💡 Hint** - Reminder to replace [name] with a unique string. e.g., "edcvr"


💻 `aws s3 mb s3://gorilla-weights-[name] --region ap-south-1`
```
make_bucket: gorilla-weights-[name]
```

# Exercise 1: Copying data between AWS regions

Each cloud provider has dedicated tools to move data between cloud regions. Let’s try transferring over the data using AWS’s built in cp command:

💻 `aws s3 cp --recursive s3://skycamp-demo-bucket/gorilla/ s3://gorilla-weights-[name]`

```
copy: s3://skycamp-demo-bucket/gorilla/pytorch_model-00002-of-00002.bin to s3://gorilla-weights-new-bucket/pytorch_model-00002-of-00002.bin
Completed 7.3 GiB/12.4 GiB (18.7 MiB/s) with 1 file(s) 
Completed 7.4 GiB/12.4 GiB (18.7 MiB/s) with 1 file(s) 
Completed 7.4 GiB/12.4 GiB (18.8 MiB/s) with 1 file(s) 
Completed 7.4 GiB/12.4 GiB (18.8 MiB/s) with 1 file(s) 
Completed 7.4 GiB/12.4 GiB (18.7 MiB/s) with 1 file(s) 
Completed 7.4 GiB/12.4 GiB (18.7 MiB/s) with 1 file(s) 
Completed 7.4 GiB/12.4 GiB (18.8 MiB/s) with 1 file(s) 
Completed 7.4 GiB/12.4 GiB (18.8 MiB/s) with 1 file(s) 
Completed 7.4 GiB/12.4 GiB (18.7 MiB/s) with 1 file(s) 
```

### This will take a long time to complete. Feel free to interrupt the command. Notice that it copies data at under 25 MiB/s.

# Exercise 2: Let's try the same transfer with skyplane 

### Configure Skyplane with AWS credentials. 

#### <span style="color:red">Choose `Y` only for AWS, and `n` for GCP and Azure.</span>

💻 `skyplane init`

```
 _____ _   ____   _______ _       ___   _   _  _____ 
/  ___| | / /\ \ / / ___ \ |     / _ \ | \ | ||  ___|
\ `--.| |/ /  \ V /| |_/ / |    / /_\ \|  \| || |__  
 `--. \    \   \ / |  __/| |    |  _  || . ` ||  __| 
/\__/ / |\  \  | | | |   | |____| | | || |\  || |___ 
\____/\_| \_/  \_/ \_|   \_____/\_| |_/\_| \_/\____/

03:37:54 [DEBUG] Found existing configuration file at /root/.skyplane/config, 
loading

(1) Configuring AWS:
    Do you want to configure AWS support in Skyplane? [Y/n]:
    Loaded AWS credentials from the AWS CLI [IAM access key ID: ...ZEXYJW]
    AWS region config file saved to /root/.skyplane/aws_config

(2) Configuring Azure:
    Do you want to configure Azure support in Skyplane? [Y/n]: n
    Disabling Azure support

(3) Configuring GCP:
    Do you want to configure GCP support in Skyplane? [Y/n]: n
    Disabling Google Cloud support

Config file saved to /root/.skyplane/config
To disable performance logging info: 
https://skyplane.org/en/latest/performance_stats_collection.html
```


> **💡 Hint** - If you run into any issues, please contact one of the Skyplane team members immediately. This step is critical to follow through the tutorial.

### Transferring with Skyplane 

💻 `skyplane cp --recursive s3://skycamp-demo-bucket/gorilla/ s3://gorilla-weights-[name]`

```
 _____ _   ____   _______ _       ___   _   _  _____ 
/  ___| | / /\ \ / / ___ \ |     / _ \ | \ | ||  ___|
\ `--.| |/ /  \ V /| |_/ / |    / /_\ \|  \| || |__  
 `--. \    \   \ / |  __/| |    |  _  || . ` ||  __| 
/\__/ / |\  \  | | | |   | |____| | | || |\  || |___ 
\____/\_| \_/  \_/ \_|   \_____/\_| |_/\_| \_/\____/

13:34:22 [WARN]  Bucket location skycamp-demo-bucket is not public. Assuming region is 
us-east-1
Logging to: /tmp/skyplane/transfer_logs/20231020_133422-b07cc30f/client.log
Using Skyplane version 0.3.2
Will transfer objects from aws:us-east-1 to aws:ap-south-1
  VMs to provision: 1x aws:ap-south-1, 1x aws:us-east-1
  Estimated egress cost: $0.02/GB
  ⠦ Querying objects for transfer...13:34:25 
  s3://skycamp-demo-bucket/gorilla/ => s3://gorilla-weights-new-bucket/ (0B)
  s3://skycamp-demo-bucket/gorilla/pytorch_model-00001-of-00002.bin => 
s3://gorilla-weights-new-bucket/pytorch_model-00001-of-00002.bin (9.26GB)
  s3://skycamp-demo-bucket/gorilla/pytorch_model-00002-of-00002.bin => 
s3://gorilla-weights-new-bucket/pytorch_model-00002-of-00002.bin (3.13GB)
Continue? [Y/n]: Y
Transfer starting (Tip: Enable auto-confirmation with `skyplane config set autoconfirm 
true`)
✓ Provisioning VMs (2/2) in 44.38s
✓ Starting gateway container on VMs (2/2) in 46.70s
⠏ Dispatching chunks... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/? bytes13:36:44 
  Transfer progress aws:ap-south-1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 GiB 411.5 MB/s 0:00:00
✓ Completing multipart uploads (2/2) in 4.86s

✅ Transfer completed successfully
Transfer runtime: 44.20s, Throughput: 2.24Gbps
```

# Exercise 3: Transferring to multiple destinations
<p style="text-align:center;">
    <img src="./assets/multicast.jpg" width=700>
</p>

In some cases, data needs to be replicated to multiple destinations. For example, say you have some freshly trained model weights: you'll want to have them accessible across multiple regions as quickly as possible. In this example, we'll show how you can run a multicast (i.e. multi-destination) transfer using Skyplane. 

## Create a secondary region bucket

Lets create a second bucket in the additional destination region `aws:eu-north-1`. 

> **💡 Hint** - Reminder to replace [name] with a unique string. e.g., "edcvr"

💻 `aws s3 mb s3://gorilla-[name] --region eu-north-1`
```
make_bucket: gorilla-[name]
```

## Running a multicast transfer using Skyplane API
To run a multicast transfer, we can simply enter a list of destinations instead of a single destination. 

In [ ]:
import skyplane 

client = skyplane.SkyplaneClient()

In [ ]:
src_bucket_path = "s3://skycamp-demo-bucket/gorilla/"
bucket_path = "s3://gorilla-weights-[name]"
another_bucket_path = "s3://gorilla-[name]"

In [ ]:
client.copy(src_bucket_path, [bucket_path, another_bucket_path], recursive=True, max_instances=1)

# Exercise 4: Cleanup 
Finally, lets use the Skyplane API to delete the buckets we created. 

In [ ]:
client.object_store().delete_bucket("gorilla-weights-[name]", provider="aws")

In [ ]:
client.object_store().delete_bucket("gorilla-[name]", provider="aws")

Run to double check that instances are all deprovisioned: 

In [ ]:
!skyplane deprovision

## 🎉 Congratulations! Your plane has now landed. Skyplane is an open sourced project. Feel free to use Skyplane for all your data mobility needs!


#### Eager to learn more? 

#### Feel free to play-around with the [Skyplane optimizer](https://optimizer.skyplane.org/), read our NSDI 2023 [paper](https://arxiv.org/abs/2210.07259), or browse through our GitHub [repository](https://github.com/skyplane-project/skyplane).

Acknowledgement: Thanks to [Skypilot](https://github.com/romilbhardwaj/skypilot-tutorial/) for the notebook template.